# Lab 7

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github//afarbin/DATA1401-Spring-2020/blob/master/Labs/Lab-7/Lab-7.ipynb)

Here are the "Gradebook" classes from lecture. For this lab, you will use these classes and are encouraged to modify them as you need.

In [0]:
import numpy as np
import math

# Create some virtual classes
class base:
    __name=""
    
    def __init__(self,name):
        self.__name=name

    def name(self):
        return self.__name

class data(base):
    def __init__(self,name):
        base.__init__(self,name)
        
class alg(base):
    def __init__(self,name):
        base.__init__(self,name)

In [0]:
#each grade is an object, with a certain corresponding gradebook name; each grade has actual properties
class grade(data):
    __value=0
    __numerical=True
    __gradebook_name=str()
    __letter_grades=["F-","F","F+","D-","D","D+","C-","C","C+","B-","B","B+","A-","A","A+"]
    
    #initialize with a name, numerical set to true, no default value
    #if there is a value, numerical is set to true or false based on value type; 
    #the value is stored as a class variable 
    def __init__(self,name,numerical=True,value=None):
        if value:
            if isinstance(value,(int,float)):
                self.__numerical=True
            elif isinstance(value,str):
                self.__numerical=False
            #modification to original code (otherwise the value wouldn't actually be stored during initialization)
            self.__value=value
    #if there's no value, numerical defaults as true
        else:            
            self.__numerical=numerical
    #gradebook_name is the name given
        self.__gradebook_name=name
    #grade has base class of 'data', with a name
        data.__init__(self,name+" Grade Algorithm")        


    def set_grade(self,value):
      #if the type of the value passed matches the set 'numerical' property, the value is stored
        if isinstance(value,(int,float)) and self.__numerical:
            self.__value=value
        elif isinstance(value,str) and not self.__numerical:
            if value in self.__letter_grades:
                self.__value=value
        else:
            print (self.name()+" Error: Bad Grade.")
            raise Exception
    
    def value(self):
        return self.__value
    
    def numerical(self):
        return self.__numerical
    
    def gradebook_name(self):
        return self.__gradebook_name
    
    def __str__(self):
        return self.__gradebook_name+": "+str(self.__value)

In [0]:
#stores student info, most importantly a dictionary of the student's grades
class student(data):
    __id_number=0
    __grades=dict()
    
    #initializes with a name and ID number
    def __init__(self,first_name, last_name,id_number):
        self.__id_number=id_number
        self.__grades=dict()
        #stems from "data" parent class
        data.__init__(self,first_name+" "+last_name+" Student Data")

    def add_grade(self,a_grade,overwrite=False):
        #works if you want to overwrite or if the corresponding 'name' of the grade (an assignment) corresponding
        #is not already in the dictionary of grades
        if overwrite or not a_grade.gradebook_name() in self.__grades:
            #the key used is that name
            self.__grades[a_grade.gradebook_name()]=a_grade
        else:
            print (self.name()+" Error Adding Grade "+a_grade.name()+". Grade already exists.")
            raise Exception

    def id_number(self):
        return self.__id_number
    
    def __getitem__(self,key):
        return self.__grades[key]

    def get_grades(self):
        return self.__grades
    
    def print_grades(self):
        for grade in self.__grades:
            print (self.__grades[grade])

In [0]:
#child 'calculator' class, that's also an 'algorithm'
class calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_grade_book):
        raise NotImplementedError

In [0]:
class grade_book(data):
    # New member class to hold arbitrary data associated with the class

    __data=dict()
    __students=dict()
    
    #initialize the grade book with a name
    def __init__(self,name):
        data.__init__(self,name+" Course Grade Book")
        self.__students=dict()
        self.__data=dict()
        
    # New method to access data
    def __getitem__(self,key):
        return self.__data[key]
            
    # New method to add data
    def __setitem__(self, key, value):
        self.__data[key] = value
        
    #saves a 'student' object, uses its ID number as a key
    def add_student(self,a_student):
        self.__students[a_student.id_number()]=a_student

    # New method to allow iterating over students
    def get_students(self):
        return self.__students

    def get_data(self):
        return self.__data
    
    #assigns a grade to a student based on the key
    def assign_grade(self,key,a_grade):
        the_student=None
        try:
            the_student=self.__students[key]
        except:
            for id in self.__students:
                if key == self.__students[id].name():
                    the_student=self.__students[id]
                    break
        if the_student:
            the_student.add_grade(a_grade)
        else:
            print (self.name()+" Error: Did not find student.")
            
    def apply_calculator(self,a_calculator,**kwargs):
        a_calculator.apply(self,**kwargs)

In [0]:
class uncurved_letter_grade_percent(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,max_grade=100.):
        self.__max_grade=max_grade
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Uncurved Percent Based Grade Calculator "+self.__grade_name+" Max="+str(self.__max_grade))
        
    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
  
        for k,a_student in a_grade_book.get_students().iteritems():
            a_grade=a_student[grade_name]

            if not a_grade.numerical():
                print (self.name()+ " Error: Did not get a numerical grade as input.")
                raise Exception
    
            percent=a_grade.value()/self.__max_grade
        
            for i,v in enumerate(self.__grades_definition):
                if percent>=v[0]:
                    break
                            
            a_student.add_grade(grade(grade_name+" Letter",value=self.__grades_definition[i][1]))
            

In [0]:
class mean_std_calculator(calculator):
    def __init__(self):
        calculator.__init__(self,"Mean and Standard Deviation Calculator")
        
    def apply(self,grade_book,grade_name,**kwargs):
        grades=list()
        for k,a_student in grade_book.get_students().items():
            grades.append(float(a_student[grade_name].value()))
        grade_book[grade_name+" Mean"] = np.mean(grades)
        grade_book[grade_name+" STD"] = math.sqrt(np.var(grades))


## CSV Reader

*Exercise 1*: The data for a class are stored in a "camma separated values" (CSV) file name `Data1401-Grades.csv` in the directory of this lab. You can see the contents using the `cat` shell command:

In [8]:
csv = open("Data1401-Grades.csv","r")
csv.readline()
csv.readline()


'1,10,7,0,10,10,8,10,10,10,14,9,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,1,9.5,15,9,9,0,9,8,0,0,0,0,0,0,0,0,0,0\n'

You will note that the first line has the names of the "columns" of data, and that subsequent lines (or "rows") have the data for each student, separated by cammas.

Recalling that in lecture we created a file reader, create a CSV reader function that takes a filename as input and returns data structure(s) that store the data in the file. Note that you are not allowed to use a library. The point here is for *you* to write the CSV reader. Some options for your data structures (pick one):

* A list of dictionaries, where each element of the list is corresponds to a row of data and the dictionaries are keyed by the column name. For example `data[5]["l3_5"]` corresponds to the 6th student's grade on lab 3 question 5.

* A list of lists (i.e. a 2-D array or matrix) and a dictionary, where each element of the "matrix" corresponds to a a specific grade for a specific student and the dictionary maps the name of the column to the column index. For example `data[5][column_names["l1_5"]]` corresponds to the 6th student's grade on lab 3 question 5.

* A dictionary of lists, where each element of the dictionary corresponds to a column of data and the lists contain the data in that column. For example `data["l3_5"][5]` corresponds to the 6th student's grade on lab 3 question 5.

* (Extra Credit) A class that simultaneously supports all of the above methods.

In [0]:
class csv_reader:
  def __init__(self, filename, structure_type="list_of_dicts"):
    #list of the fields
    csv = open(filename,"r")
    fields = (csv.readline().rstrip()).split(",")
    
    #separate list of lists of student data
    grades = list()
    while csv.readline().rstrip():
      grades.append((csv.readline().rstrip()).split(","))

    if structure_type == "list_of_dicts":
      data = list()
      for student in range(len(grades)):
        temp = dict()
        for field in range(len(fields)):
          temp[fields[field]]=grades[student][field]
        data.append(temp)

    elif structure_type == "list_of_lists":
      data = grades
      self.__field_indices = {fields[index]:index for index in range(len(fields))}

    elif structure_type == "dict_of_lists": 
      data = dict()
      for field in range(len(fields)):
        temp = [grades[student][field] for student in range(len(grades))]
        data[fields[field]] = temp
      
    else:
      raise AttributeError("Invalid structure type.")

    self.__fields = fields
    self.__grades = grades
    self.__structure_type = structure_type
    self.__data = data
  
  def __getitem__(self,value):
    return self.__data[value]
    
  def get_type(self):
    return self.__structure_type
  
  def get_fields(self):
    return self.__fields
  
  def get_grades(self):
    return self.__grades

  def get_data(self):
    return self.__data
  
  #only does something if the object is a list of lists (and thus has accompanying dictionary of field indices)
  def get_field_indices(self):
    if isinstance(self.__field_indices,dict):
      return self.__field_indices
    else:
      raise NotImplementedError

  def display(self):
    print(self.__fields)
    for points in self.__grades:
      print(points)

In [10]:
a = csv_reader("Data1401-Grades.csv", "list_of_dicts")
b = csv_reader("Data1401-Grades.csv", "list_of_lists")
c = csv_reader("Data1401-Grades.csv", "dict_of_lists")
print(a[3]["l3_1"])
print(b[3][b.get_field_indices()["l3_1"]])
print(c["l3_1"][3])
a.display()


10
10
10
['l1_n', 'l1_1', '12_n', 'l2_1', 'l2_2', 'l2_3', 'l2_4', 'l2_5', 'l2_6', 'l2_7', 'l3_n', 'l3_1', 'l3_2', 'l3_3', 'l3_4', 'l3_5', 'l3_6', 'l3_7', 'l3_8', 'l3_9', 'l3_10', 'l3_11', 'l3_12', 'l3_13', 'l3_14', 'l4_n', 'l4_1', 'l4_2', 'l4_3', 'l4_4', 'l4_5', 'l4_6', 'l4_7', 'l4_8', 'l4_9', 'l4_10', 'l4_11', 'q1_n', 'q1_1', 'e1_n', 'e1_1', 'e1_2', 'e1_3', 'e1_4', 'e1_5', 'e1_6', 'e1_7', 'e1_8', 'e1_9', 'e1_10', 'e1_11', 'e1_12', 'e1_13', 'e1_14', 'e1_15']
['1', '10', '7', '0', '0', '0', '0', '0', '0', '0', '14', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '11', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '15', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
['1', '10', '7', '10', '10', '3', '9.5', '10', '10', '9.5', '14', '10', '10', '10', '8', '5', '10', '5', '10', '3', '0', '10', '3', '10', '8', '11', '10', '10', '10', '10', '10', '10', '0', '0', '10', '5', '0', '1', '10', '15', '9', '9', '10', '9', '7', '9'

## Creating a Gradebook

*Exercise 2:* In lecture we used pandas to read the CSV file and create the grade book. The example below works for the CSV file for this lab. Modify the code below to use your CSV reader instead.

In [0]:
# import pandas as pd
# class_data=pd.read_csv("Data1401-Grades.csv", )

# a_grade_book=grade_book("Data 1401")

# for student_i in range(class_data.shape[0]):
#     a_student_0=student("Student",str(student_i),student_i)

#     for k in class_data.keys():
#         a_student_0.add_grade(grade(k,value=class_data[k][student_i]))

#     a_grade_book.add_student(a_student_0)


In [0]:
class_data = csv_reader("Data1401-Grades.csv", "dict_of_lists")
a_grade_book=grade_book("Data 1401")

#this implementation is specifically for the dictionary of lists structure
for student_i in range(len(class_data.get_grades())) :
    temp_student = student("Student",str(student_i),student_i)

    for fields in class_data.get_fields():
      temp_student.add_grade(grade(fields,value=float(class_data[fields][student_i])))
  
    a_grade_book.add_student(temp_student)

In [12]:
print((a_grade_book.get_students()[1]).id_number())
a_grade_book.get_students()

1


{0: <__main__.student at 0x7f53c9e81dd8>,
 1: <__main__.student at 0x7f53c9e2f550>,
 2: <__main__.student at 0x7f53c9e331d0>,
 3: <__main__.student at 0x7f53c9e33e10>,
 4: <__main__.student at 0x7f53c9e3aa90>,
 5: <__main__.student at 0x7f53c9e406d8>,
 6: <__main__.student at 0x7f53c9e47358>,
 7: <__main__.student at 0x7f53c9e47f98>}

## Grade Summing

*Exercise 3:* In lectre we will change the design of our algorithm classes and then update the `uncurved_letter_grade_percent` calculator. In lecture we also created a `grade_summer` calcuator that takes a prefix (for example `e1_` and a number `n`) and sums all grades starting with that prefix up to `n` and creates a new sum grade. Update this calculator (below) to the new design of our algorithm classes. Test your updated calculator by using it to sum the grades for all labs, quizzes, and exams of each student.

In [0]:
class grade_summer(calculator):
    def __init__(self,prefix,n=100):
        self.__prefix=prefix
        self.__n=n
        calculator.__init__(self,"Grade Sum Calculator")
        
    def apply(self,a_grade_book):
      #iterate over every student
      for index,a_student in a_grade_book.get_students().items(): 
        grade_names = list()
        #list of grade names
        for i in range(1,(self.__n)+1):
          try:
            a_student[self.__prefix + str(i)]
            grade_names.append(self.__prefix + str(i))
          except:
            break
        #calculating the sum of points for the grade
        grade_sum = 0  
        for grade_name in grade_names:
            grade_sum+=float(a_student[grade_name].value())
        a_student.add_grade(grade(self.__prefix + " Sum", value=float(grade_sum)))

In [14]:
prefixes = ["l1_","l2_","l3_","l4_","q1_","e1_"]

for prefix in prefixes:
  temp_summer = grade_summer(prefix)
  a_grade_book.apply_calculator(temp_summer)

#print info
for a_student in range(len((class_data.get_grades()))):
  print("Student " + str(a_student+1) + " grades: " )
  (a_grade_book.get_students()[a_student]).print_grades()

Student 1 grades: 
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 0
l2_3: 0
l2_4: 0
l2_5: 0
l2_6: 0
l2_7: 0
l3_n: 14.0
l3_1: 0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0
q1_1: 0
e1_n: 15.0
e1_1: 0
e1_2: 0
e1_3: 0
e1_4: 0
e1_5: 0
e1_6: 0
e1_7: 0
e1_8: 0
e1_9: 0
e1_10: 0
e1_11: 0
e1_12: 0
e1_13: 0
e1_14: 0
e1_15: 0
l1_ Sum: 10.0
l2_ Sum: 0
l3_ Sum: 0
l4_ Sum: 0
q1_ Sum: 0
e1_ Sum: 0
Student 2 grades: 
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 10.0
l2_2: 10.0
l2_3: 3.0
l2_4: 9.5
l2_5: 10.0
l2_6: 10.0
l2_7: 9.5
l3_n: 14.0
l3_1: 10.0
l3_2: 10.0
l3_3: 10.0
l3_4: 8.0
l3_5: 5.0
l3_6: 10.0
l3_7: 5.0
l3_8: 10.0
l3_9: 3.0
l3_10: 0
l3_11: 10.0
l3_12: 3.0
l3_13: 10.0
l3_14: 8.0
l4_n: 11.0
l4_1: 10.0
l4_2: 10.0
l4_3: 10.0
l4_4: 10.0
l4_5: 10.0
l4_6: 10.0
l4_7: 0
l4_8: 0
l4_9: 10.0
l4_10: 5.0
l4_11: 0
q1_n: 1.0
q1_1: 10.0
e1_n: 15.0


## Curving Grades

*Exercise 4:* Use the `mean_std_calculator` above to calculate the mean and standard deviation for every lab, quiz, and exam in the class. Add a new print function to the `grade_book` class to print out such information in a nice way, and use this function to show your results.


In [0]:
class student(data):
    __id_number=0
    __grades=dict()
    
    def __init__(self,first_name, last_name,id_number):
        self.__id_number=id_number
        self.__grades=dict()
        data.__init__(self,first_name+" "+last_name+" Student Data")

    def add_grade(self,a_grade,overwrite=False):
        if overwrite or not a_grade.gradebook_name() in self.__grades:
            self.__grades[a_grade.gradebook_name()]=a_grade
        else:
            print (self.name()+" Error Adding Grade "+a_grade.name()+". Grade already exists.")
            raise Exception

    def id_number(self):
        return self.__id_number
    
    def __getitem__(self,key):
        return self.__grades[key]

    def get_grades(self):
        return self.__grades
    
    def print_grades(self):
        for grade in self.__grades:
            print (self.__grades[grade])

In [0]:
class grade_book(data):

    __data=dict()
    __students=dict()
    
    def __init__(self,name):
        data.__init__(self,name+" Grade Book")
        self.__students=dict()
        self.__data=dict()
        
    def __getitem__(self,key):
        return self.__data[key]
            
    def __setitem__(self, key, value):
        self.__data[key] = value
        
    def add_student(self,a_student):
        self.__students[a_student.id_number()]=a_student

    def get_students(self):
        return self.__students

    def get_data(self):
        return self.__data

    def assign_grade(self,key,a_grade):
        the_student=None
        try:
            the_student=self.__students[key]
        except:
            for id in self.__students:
                if key == self.__students[id].name():
                    the_student=self.__students[id]
                    break
        if the_student:
            the_student.add_grade(a_grade)
        else:
            print (self.name()+" Error: Did not find student.")
            
    def apply_calculator(self,a_calculator,**kwargs):
        a_calculator.apply(self,**kwargs)

    #new print function for the gradebook data
    def print_data(self):
        print(self.name() + " Data: ")
        for key,data in (self.__data).items():
          #if "Mean" appears in the grade name -> information about a new grade
          if "Mean" in key:
            print("_____________________")
          print(key + ": " + str(data))

In [17]:
#re-instantiating a_grade_book so it has access to 'print_data'
a_grade_book=grade_book("Data 1401")
prefixes = ["l1_","l2_","l3_","l4_","q1_","e1_"]

for student_i in range(len(class_data.get_grades())):
  temp_student = student("Student",str(student_i),student_i)
  for fields in class_data.get_fields():
    temp_student.add_grade(grade(fields,value=float(class_data[fields][student_i])))
  a_grade_book.add_student(temp_student)

for prefix in prefixes:
  temp_summer = grade_summer(prefix)
  a_grade_book.apply_calculator(temp_summer)

  

#applying the calculator
statcalc = mean_std_calculator()
for prefix in prefixes:
  name = prefix+" Sum"
  a_grade_book.apply_calculator(statcalc, grade_name=name)
  
a_grade_book.print_data()

Data 1401 Grade Book Data: 
_____________________
l1_ Sum Mean: 10.0
l1_ Sum STD: 0.0
_____________________
l2_ Sum Mean: 54.625
l2_ Sum STD: 21.55914829022705
_____________________
l3_ Sum Mean: 71.875
l3_ Sum STD: 40.50520799847842
_____________________
l4_ Sum Mean: 39.625
l4_ Sum STD: 21.371344716699507
_____________________
q1_ Sum Mean: 7.3125
q1_ Sum STD: 4.227421643271463
_____________________
e1_ Sum Mean: 94.875
e1_ Sum STD: 39.107024624739736


*Exercise 5:* In lecture we will change the design of our algorithms classes and then update the `uncurved_letter_grade_percent` calculator. Do the same for the `curved_letter_grade` calculator below and by curving all the lab, quiz, and exam grades.

In [0]:
class curved_letter_grade(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,mean,std,max_grade=100.):
        self.__max_grade=max_grade
        self.__mean=mean
        self.__std=std
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Curved Percent Grade Calculator for "+self.__grade_name+ \
                                  " Mean="+str(self.__mean)+\
                                  " STD="+str(self.__std)+\
                                  " Max="+str(self.__max_grade))

    def apply(self,a_grade_book):
      for index,a_student in a_grade_book.get_students().items():
        if not isinstance(a_student[self.__grade_name],grade):
            print(self.name()+ " Error: Did not get an proper grade as input.")
            raise Exception
        if not a_student[self.__grade_name].numerical():
            print(self.name()+ " Error: Did not get a numerical grade as input.")
            raise Exception
        

        percent=a_student[self.__grade_name].value()/self.__max_grade
        shift_to_zero=percent-(self.__mean/self.__max_grade)
        # +0.0001 to prevent error when STD = 0 (l1_ grades)
        scale_std=0.1*shift_to_zero/((self.__std/self.__max_grade)+0.0001) 
        scaled_percent=scale_std+0.8


        for i,v in enumerate(self.__grades_definition):
            if scaled_percent>=v[0]:
                break
                
        scaled_grade = grade(self.__grade_name + " Curved",value=self.__grades_definition[i][1])
        a_student.add_grade(scaled_grade)

In [19]:
prefixes = ["l1_","l2_","l3_","l4_","q1_","e1_"]
max_grades = [10.0,70.0,140.0,110.,10.0,150.0]

#apply calculator
index = 0
for prefix in prefixes:
  name = prefix+ " Sum"
  curve_calc = curved_letter_grade(name, a_grade_book[name + " Mean"], a_grade_book[name + " STD"], max_grade=max_grades[index])
  a_grade_book.apply_calculator(curve_calc)
  index+=1

#print out info
for student in range(len((class_data.get_grades()))):
  print("Student " + str(student+1) + " grades: " )
  (a_grade_book.get_students()[student]).print_grades()

Student 1 grades: 
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 0
l2_3: 0
l2_4: 0
l2_5: 0
l2_6: 0
l2_7: 0
l3_n: 14.0
l3_1: 0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0
q1_1: 0
e1_n: 15.0
e1_1: 0
e1_2: 0
e1_3: 0
e1_4: 0
e1_5: 0
e1_6: 0
e1_7: 0
e1_8: 0
e1_9: 0
e1_10: 0
e1_11: 0
e1_12: 0
e1_13: 0
e1_14: 0
e1_15: 0
l1_ Sum: 10.0
l2_ Sum: 0
l3_ Sum: 0
l4_ Sum: 0
q1_ Sum: 0
e1_ Sum: 0
l1_ Sum Curved: B-
l2_ Sum Curved: F
l3_ Sum Curved: C-
l4_ Sum Curved: C-
q1_ Sum Curved: C-
e1_ Sum Curved: F
Student 2 grades: 
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 10.0
l2_2: 10.0
l2_3: 3.0
l2_4: 9.5
l2_5: 10.0
l2_6: 10.0
l2_7: 9.5
l3_n: 14.0
l3_1: 10.0
l3_2: 10.0
l3_3: 10.0
l3_4: 8.0
l3_5: 5.0
l3_6: 10.0
l3_7: 5.0
l3_8: 10.0
l3_9: 3.0
l3_10: 0
l3_11: 10.0
l3_12: 3.0
l3_13: 10.0
l3_14: 8.0
l4_n: 11.0
l4_1: 10.0
l4_2: 10.0
l4_3: 10.0


## Final Course Grade

*Exercise 6:* Write a new calculator that sums grades with a prefix, as in the `grade_summer` calculator, but drops `n` lowest grades. Apply the algorithm to drop the lowest lab grade in the data.


In [0]:
#takes a gradebook of students and drops a certain number of lowest grades for each student
class cutoff_grade_sum(calculator):
  def __init__(self,prefix_list,max_grades,cutoff=0,n=100):
    self.__prefix_list = prefix_list
    self.__max_grades = max_grades
    self.__cutoff = cutoff
    self.__n = n
    calculator.__init__(self,"Cutoff Grade Sum Calculator for " + str(self.__prefix_list))


  def apply(self,a_grade_book):
    for index,a_student in a_grade_book.get_students().items():
      #build dictionary of all grade names
      grade_dict = dict()
      for prefix in self.__prefix_list:
        temp_list = list()
        for i in range(1, (self.__n)+1):
          try:
            a_student[prefix + str(i)]
            temp_list.append(prefix + str(i))
          except:
            break
        grade_dict[prefix] = temp_list

      #build a dictionary of summed grades
      grade_sums_dict = dict()
      for key,grade_list in grade_dict.items():
        temp_sum = 0
        for a_grade in grade_list:
          temp_sum += float(a_student[a_grade].value())
        grade_sums_dict[key] = temp_sum

      #build a dictionary of scaled grades (scaled by ratio to the appropriate max grade)
      scaled_grade_sums_dict = dict()
      for key,summed_grade in grade_sums_dict.items():
          scaled_grade_sums_dict[key] = summed_grade/self.__max_grades[key]
          
      #drops 'cutoff' (n) number of lowest grades based on the percentages in scaled_grade_sums 
      for drop_counter in range(self.__cutoff):
        lowest_grade_key = min(scaled_grade_sums_dict, key=scaled_grade_sums_dict.get)
        grade_sums_dict[lowest_grade_key] = "Dropped"
        a_student.add_grade(grade(lowest_grade_key + " Sum Curved", value="Dropped"),overwrite=True)
        scaled_grade_sums_dict.pop(lowest_grade_key)

      #stores grade sums in the student object
      for key,summed_grade in grade_sums_dict.items():
        a_student.add_grade(grade(key + " Sum", value=summed_grade), overwrite=True)

In [21]:
max_grades = {"l1_":10, "l2_":70, "l3_":140, "l4_":110}
prefixes = ["l1_","l2_","l3_","l4_"]
cutoff_calc = cutoff_grade_sum(prefixes,max_grades,cutoff=1)
a_grade_book.apply_calculator(cutoff_calc)

for student in range(len((class_data.get_grades()))):
  print("Student " + str(student+1) + " grades: " )
  (a_grade_book.get_students()[student]).print_grades()

Student 1 grades: 
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 0
l2_3: 0
l2_4: 0
l2_5: 0
l2_6: 0
l2_7: 0
l3_n: 14.0
l3_1: 0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0
q1_1: 0
e1_n: 15.0
e1_1: 0
e1_2: 0
e1_3: 0
e1_4: 0
e1_5: 0
e1_6: 0
e1_7: 0
e1_8: 0
e1_9: 0
e1_10: 0
e1_11: 0
e1_12: 0
e1_13: 0
e1_14: 0
e1_15: 0
l1_ Sum: 10.0
l2_ Sum: Dropped
l3_ Sum: 0
l4_ Sum: 0
q1_ Sum: 0
e1_ Sum: 0
l1_ Sum Curved: B-
l2_ Sum Curved: Dropped
l3_ Sum Curved: C-
l4_ Sum Curved: C-
q1_ Sum Curved: C-
e1_ Sum Curved: F
Student 2 grades: 
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 10.0
l2_2: 10.0
l2_3: 3.0
l2_4: 9.5
l2_5: 10.0
l2_6: 10.0
l2_7: 9.5
l3_n: 14.0
l3_1: 10.0
l3_2: 10.0
l3_3: 10.0
l3_4: 8.0
l3_5: 5.0
l3_6: 10.0
l3_7: 5.0
l3_8: 10.0
l3_9: 3.0
l3_10: 0
l3_11: 10.0
l3_12: 3.0
l3_13: 10.0
l3_14: 8.0
l4_n: 11.0
l4_1: 10.0
l4_2: 10.0

*Exercise 7*: Write a new calculator that creates a new letter grade based on a weighted average of letter grades, by assigning the following numerical values to letter grades:

In [0]:
class final_grade_calculator(calculator):
  __grade_map = {"A+":12,
          "A":11,
          "A-":10,
          "B+":9,
          "B":8,
          "B-":7,
          "C+":6,
          "C":5,
          "C-":4,
          "D+":3,
          "D":2,
          "D-":1,
          "F":0}
  __max_weight = __grade_map["A+"]

  def __init__(self,weights):
    self.__weights = weights
    calculator.__init__(self, "Final Grade Calculator with Weights = " + str(weights))

  def apply(self,a_grade_book):
    for a_student_name,a_student in a_grade_book.get_students().items():
      #loop through the grades of every student for each prefix 
      grade_dict = dict()
      for prefix,weight in self.__weights.items():
        temp_sum = 0
        counter = 0
        #adds grade to temp_sum if it contains the correct prefix and is a letter grade
        for a_grade_name,a_grade in a_student.get_grades().items():
          if (a_grade_name[0]==prefix) and (a_grade.value()!="Dropped") and (not a_grade.numerical()):
            temp_sum += self.__grade_map[a_grade.value()]
            counter += 1
        #stores average value of each kind of grade (lab, quiz, exam) so final grade can be calculated
        grade_dict[prefix] = (temp_sum/counter)

      #calcuating final grade by using the weights
      final_grade = 0
      for prefix,scaled_sum in grade_dict.items():
        final_grade += scaled_sum*self.__weights[prefix]
    
      #storing final grade
      for prefix,a_grade in self.__grade_map.items():
        if final_grade >= a_grade:
          a_student.add_grade(grade("Final Grade",value=prefix),overwrite = True)
          break

Test you calculator by applying the weights from the syllabus of this course and computing everyone's grade in the course.

In [23]:
weights = {"l":0.5, "q":0.1, "e":0.4}
final_calc = final_grade_calculator(weights)
a_grade_book.apply_calculator(final_calc)

#print out info
for student in range(len((class_data.get_grades()))):
  print("Student " + str(student+1) + " grades: " )
  (a_grade_book.get_students()[student]).print_grades()

Student 1 grades: 
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 0
l2_2: 0
l2_3: 0
l2_4: 0
l2_5: 0
l2_6: 0
l2_7: 0
l3_n: 14.0
l3_1: 0
l3_2: 0
l3_3: 0
l3_4: 0
l3_5: 0
l3_6: 0
l3_7: 0
l3_8: 0
l3_9: 0
l3_10: 0
l3_11: 0
l3_12: 0
l3_13: 0
l3_14: 0
l4_n: 11.0
l4_1: 0
l4_2: 0
l4_3: 0
l4_4: 0
l4_5: 0
l4_6: 0
l4_7: 0
l4_8: 0
l4_9: 0
l4_10: 0
l4_11: 0
q1_n: 1.0
q1_1: 0
e1_n: 15.0
e1_1: 0
e1_2: 0
e1_3: 0
e1_4: 0
e1_5: 0
e1_6: 0
e1_7: 0
e1_8: 0
e1_9: 0
e1_10: 0
e1_11: 0
e1_12: 0
e1_13: 0
e1_14: 0
e1_15: 0
l1_ Sum: 10.0
l2_ Sum: Dropped
l3_ Sum: 0
l4_ Sum: 0
q1_ Sum: 0
e1_ Sum: 0
l1_ Sum Curved: B-
l2_ Sum Curved: Dropped
l3_ Sum Curved: C-
l4_ Sum Curved: C-
q1_ Sum Curved: C-
e1_ Sum Curved: F
Final Grade: D
Student 2 grades: 
l1_n: 1.0
l1_1: 10.0
12_n: 7.0
l2_1: 10.0
l2_2: 10.0
l2_3: 3.0
l2_4: 9.5
l2_5: 10.0
l2_6: 10.0
l2_7: 9.5
l3_n: 14.0
l3_1: 10.0
l3_2: 10.0
l3_3: 10.0
l3_4: 8.0
l3_5: 5.0
l3_6: 10.0
l3_7: 5.0
l3_8: 10.0
l3_9: 3.0
l3_10: 0
l3_11: 10.0
l3_12: 3.0
l3_13: 10.0
l3_14: 8.0
l4_n: 11.0
l4_1: 